In [26]:
import pandas as pd

In [ ]:
df1 = pd.read_csv('../data/grades.csv')

In [211]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104 entries, 0 to 103
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ID      104 non-null    int64  
 1   Name    104 non-null    object 
 2   GPA     104 non-null    float64
 3   Major   104 non-null    object 
dtypes: float64(1), int64(1), object(2)
memory usage: 3.4+ KB


In [213]:
df1.head()

,ID,Name,GPA,Major
0,23,Rose Rios,3.97,Economics
1,55,Ian Brown,1.44,Political Science
2,54,Stanley Dean,3.75,Philosophy
3,19,Stephanie Henderson,1.86,Mathematics
4,81,Angela Mora,3.86,Physics


In [215]:
df1.tail()

,ID,Name,GPA,Major
99,22,Lisa Chen,1.45,Art
100,96,Jamie Knight,1.65,Business
101,95,Karen Williams,1.16,Business
102,80,Carrie Trujillo MD,1.44,Theatre
103,58,Renee Austin,2.60,Business


In [217]:
df1.duplicated().sum()

4

In [219]:
df1[df1.duplicated()]

,ID,Name,GPA,Major
100,96,Jamie Knight,1.65,Business
101,95,Karen Williams,1.16,Business
102,80,Carrie Trujillo MD,1.44,Theatre
103,58,Renee Austin,2.60,Business


In [221]:
df1.drop_duplicates(inplace=True)

In [223]:
print(df1.isnull().sum())

ID       0
Name     0
GPA      0
Major    0
dtype: int64


In [225]:
df2 = pd.read_excel('../data/tuition.xlsx')

In [227]:
df2[df2.duplicated()]

,ID,Name,State Resident,Tuition,Financial Aid
100,4,Eric Ellis,No,30000.0,12978


In [170]:
df2.drop_duplicates(inplace=True)

In [229]:
df2[df2.duplicated()]

,ID,Name,State Resident,Tuition,Financial Aid
100,4,Eric Ellis,No,30000.0,12978


In [231]:
print(df2.isnull().sum())
df2.dropna(inplace=True)
print(df2.isnull().sum())
df2.to_csv('testme1.csv')
df1.to_csv('testme2.csv')

ID                0
Name              0
State Resident    2
Tuition           2
Financial Aid     0
dtype: int64
ID                0
Name              0
State Resident    0
Tuition           0
Financial Aid     0
dtype: int64


In [233]:
#Inner join
df3 = pd.merge(df1, df2, on='ID', how='inner')
print(df3.isnull().sum())

#inner_merged_df.to_csv('testing.csv', index=False)

ID                0
Name_x            0
GPA               0
Major             0
Name_y            0
State Resident    0
Tuition           0
Financial Aid     0
dtype: int64


In [235]:
df3

,ID,Name_x,GPA,Major,Name_y,State Resident,Tuition,Financial Aid
0,23,Rose Rios,3.97,Economics,Rose Rios,Yes,20000.0,11038
1,55,Ian Brown,1.44,Political Science,Ian Brown,Yes,20000.0,6977
2,54,Stanley Dean,3.75,Philosophy,Stanley Dean,Yes,20000.0,11481
3,81,Angela Mora,3.86,Physics,Angela Mora,Yes,20000.0,12031
4,60,Mark Martin,1.47,Chemistry,Mark Martin,Yes,20000.0,12477
...,...,...,...,...,...,...,...,...
94,41,Brenda Valentine,3.10,Physics,Brenda Valentine,No,30000.0,9915
95,86,Grace Hoffman,1.30,Chemistry,Grace Hoffman,No,30000.0,12102
96,10,Joshua Ware,3.29,English,Joshua Ware,No,30000.0,11174
97,61,Ms. Samantha Steele,2.56,Spanish,Samantha Steele,No,30000.0,13103


In [237]:
df3.rename(columns={'Name_x': 'Full Name'}, inplace=True)
df3.drop('Name_y', axis=1, inplace=True)
df3.head()



,ID,Full Name,GPA,Major,State Resident,Tuition,Financial Aid
0,23,Rose Rios,3.97,Economics,Yes,20000.0,11038
1,55,Ian Brown,1.44,Political Science,Yes,20000.0,6977
2,54,Stanley Dean,3.75,Philosophy,Yes,20000.0,11481
3,81,Angela Mora,3.86,Physics,Yes,20000.0,12031
4,60,Mark Martin,1.47,Chemistry,Yes,20000.0,12477


In [239]:
df3[['First Name', 'Last Name']] = df3['Full Name'].str.split(n=1, expand=True)
df3.head()

,ID,Full Name,GPA,Major,State Resident,Tuition,Financial Aid,First Name,Last Name
0,23,Rose Rios,3.97,Economics,Yes,20000.0,11038,Rose,Rios
1,55,Ian Brown,1.44,Political Science,Yes,20000.0,6977,Ian,Brown
2,54,Stanley Dean,3.75,Philosophy,Yes,20000.0,11481,Stanley,Dean
3,81,Angela Mora,3.86,Physics,Yes,20000.0,12031,Angela,Mora
4,60,Mark Martin,1.47,Chemistry,Yes,20000.0,12477,Mark,Martin


In [244]:
df3['GPA Rank'] = df3['GPA'].rank()
df3['GPA Percentile'] = df3['GPA Rank'] / df1['GPA'].count()
df3.head()

,ID,Full Name,GPA,Major,State Resident,Tuition,Financial Aid,First Name,Last Name,GPA Rank,GPA Percentile
0,23,Rose Rios,3.97,Economics,Yes,20000.0,11038,Rose,Rios,99.0,0.990
1,55,Ian Brown,1.44,Political Science,Yes,20000.0,6977,Ian,Brown,15.5,0.155
2,54,Stanley Dean,3.75,Philosophy,Yes,20000.0,11481,Stanley,Dean,94.5,0.945
3,81,Angela Mora,3.86,Physics,Yes,20000.0,12031,Angela,Mora,97.0,0.970
4,60,Mark Martin,1.47,Chemistry,Yes,20000.0,12477,Mark,Martin,18.0,0.180
